# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
# %pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

In [2]:
# %pip install -U scikit-learn

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [3]:
# Data loading functions. Uncomment the one you want to use
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from worcliver.load_data import load_data
from function_files.self_made import load_D_set, return_X_y, removal_zero_var, create_cross_val_sets


df = load_data()

# Data clean-up (before splitting data)
In this section double entries, such as repeated features and samples, will be removed from the dataset.

In [4]:
'''
These 2 lines add a duplicate sample in the DataFrame, you can than observe what the effect is 
in the printed sentence. If you'd like to see it indeed works, please uncomment these lines.
'''
# row = df.iloc[0].copy()
# df = df.append(row, ignore_index=True)
'''
In the lines below we first drop all duplicate entries, whether it is a duplicate row or a column. 
Additionally, the number of duplicates will be returned to the user.
'''

df_f = df.drop_duplicates()
print(f'The number of samples (after removal of duplicates): {len(df_f.index)}, the number of duplicates that had to be removed: {len(df.index)-len(df_f.index)}')
print(f'The number of columns (after removal of duplicates): {len(df_f.columns)}, the number of duplicates that had to be removed: {len(df.columns)-len(df_f.columns)}')

The number of samples (after removal of duplicates): 186, the number of duplicates that had to be removed: 0
The number of columns (after removal of duplicates): 494, the number of duplicates that had to be removed: 0


In [5]:
df = df_f                                       # So we can just use df from now on
df = df.replace('', np.nan)                     # Any empty values are replaced for NaN
df.describe(include='all')                    # Uncomment if interested

,label,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,PREDICT_original_sf_cvar_std_2.5D,PREDICT_original_sf_prax_avg_2.5D,PREDICT_original_sf_prax_std_2.5D,PREDICT_original_sf_evar_avg_2.5D,PREDICT_original_sf_evar_std_2.5D,PREDICT_original_sf_solidity_avg_2.5D,PREDICT_original_sf_solidity_std_2.5D,PREDICT_original_sf_area_avg_2.5D,PREDICT_original_sf_area_max_2.5D,PREDICT_original_sf_area_min_2.5D,PREDICT_original_sf_area_std_2.5D,PREDICT_original_sf_volume_2.5D,PREDICT_original_of_theta_x,PREDICT_original_of_theta_y,PREDICT_original_of_theta_z,PREDICT_original_hf_min,PREDICT_original_hf_max,PREDICT_original_hf_mean,PREDICT_original_hf_median,PREDICT_original_hf_std,PREDICT_original_hf_skewness,PREDICT_original_hf_kurtosis,PREDICT_original_hf_peak,PREDICT_original_hf_peak_position,PREDICT_original_hf_range,PREDICT_original_hf_energy,PREDICT_original_hf_quartile_range,PREDICT_original_hf_entropy,PREDICT_original_logf_min_sigma1,PREDICT_original_logf_max_sigma1,...,"PREDICT_original_vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0",PREDICT_original_phasef_monogenic_min_WL3_N5,PREDICT_original_phasef_monogenic_max_WL3_N5,PREDICT_original_phasef_monogenic_mean_WL3_N5,PREDICT_original_phasef_monogenic_median_WL3_N5,PREDICT_original_phasef_monogenic_std_WL3_N5,PREDICT_original_phasef_monogenic_skewness_WL3_N5,PREDICT_original_phasef_monogenic_kurtosis_WL3_N5,PREDICT_original_phasef_monogenic_peak_WL3_N5,PREDICT_original_phasef_monogenic_peak_position_WL3_N5,PREDICT_original_phasef_monogenic_range_WL3_N5,PREDICT_original_phasef_monogenic_energy_WL3_N5,PREDICT_original_phasef_monogenic_quartile_range_WL3_N5,PREDICT_original_phasef_monogenic_entropy_WL3_N5,PREDICT_original_phasef_phasecong_min_WL3_N5,PREDICT_original_phasef_phasecong_max_WL3_N5,PREDICT_original_phasef_phasecong_mean_WL3_N5,PREDICT_original_phasef_phasecong_median_WL3_N5,PREDICT_original_phasef_phasecong_std_WL3_N5,PREDICT_original_phasef_phasecong_skewness_WL3_N5,PREDICT_original_phasef_phasecong_kurtosis_WL3_N5,PREDICT_original_phasef_phasecong_peak_WL3_N5,PREDICT_original_phasef_phasecong_peak_position_WL3_N5,PREDICT_original_phasef_phasecong_range_WL3_N5,PREDICT_original_phasef_phasecong_energy_WL3_N5,PREDICT_original_phasef_phasecong_quartile_range_WL3_N5,PREDICT_original_phasef_phasecong_entropy_WL3_N5,PREDICT_original_phasef_phasesym_min_WL3_N5,PREDICT_original_phasef_phasesym_max_WL3_N5,PREDICT_original_phasef_phasesym_mean_WL3_N5,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
count,186,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,1.860000e+02,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,1.860000e+02,186.000000,186.000000,186.000000,186.000000,...,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,1.860000e+02,186.000000,186.000000,186.0,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.0,186.0,186.000000,186.000000,186.000000,186.000000,186.0,186.000000,186.000000,186.000000,186.000000

### Conclusion
The table above shows there are only 2 labels present in the dataset, from observation of the csv.file we can see the labels are 'malignant' and 'benigne'. What is also stated here, that 94 of the 186 samples was labeled malignant (and thus 92 samples are labeled benigne.) This information is usefull in the next step, to determine whether a stratified splitting of our data, would be necessary.

# Data splitting
In this section the dataset will be split into two datasets, the Design set (D_set) and the Final Test set (Ft_set). This splitting will be performed randomly where 80% will end up in D_set and the remaining 20% will be stored in Ft_set. Ft_set will not be used at all, until the very latest to evaluate our tool and establish how generalising the tool is. In the initial split we will randomly split the dataset, as both labels are equally abundant in the set.

In [6]:
from sklearn.model_selection import train_test_split
# D_set, Ft_set = train_test_split(df, test_size=0.2)
# D_set = pd.DataFrame(D_set)
# Ft_set = pd.DataFrame(Ft_set)                         

In [7]:
# D_set.to_csv("C:/Users/dgjpa/D_set.csv")
# Ft_set = Ft_set.to_csv("C:/Users/dgjpa/Ft_set.csv")

'''
These lines were used to save the splitted sets, to make sure Design and Test set won't alter.
''' 

"\nThese lines were used to save the splitted sets, to make sure Design and Test set won't alter.\n"

# Data splitting (2/2)
First, load_sets.py there will be two functions who returns a dataframe from the save .csv file. D_usable and Ft_usable are both DataFrames including both the labels as the feature information, and are the resulting datasets from the initial split as can be seen above.

X_d is a Dataframe including all the information about a case but with the labels removed, whereas y_d is a DataFrame including only the labels.

### !!! Important v
Still unclear: Is whether it is necessary to manually split the X_d & y_d DataFrames already in X_train/y_train and X_test/y_test sets

This second split is ofcourse necessary to perform cross-validation, however several sklearn-functions seem to have this split already incorporated in their functions, such as: cross_validate, cross_val_score(Seems to be especially handy)

In [ ]:
D_s = load_D_set()
D_s.head()

,label,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,PREDICT_original_sf_cvar_std_2.5D,PREDICT_original_sf_prax_avg_2.5D,PREDICT_original_sf_prax_std_2.5D,PREDICT_original_sf_evar_avg_2.5D,PREDICT_original_sf_evar_std_2.5D,PREDICT_original_sf_solidity_avg_2.5D,PREDICT_original_sf_solidity_std_2.5D,PREDICT_original_sf_area_avg_2.5D,PREDICT_original_sf_area_max_2.5D,PREDICT_original_sf_area_min_2.5D,PREDICT_original_sf_area_std_2.5D,PREDICT_original_sf_volume_2.5D,PREDICT_original_of_theta_x,PREDICT_original_of_theta_y,PREDICT_original_of_theta_z,PREDICT_original_hf_min,PREDICT_original_hf_max,PREDICT_original_hf_mean,PREDICT_original_hf_median,PREDICT_original_hf_std,PREDICT_original_hf_skewness,PREDICT_original_hf_kurtosis,PREDICT_original_hf_peak,PREDICT_original_hf_peak_position,PREDICT_original_hf_range,PREDICT_original_hf_energy,PREDICT_original_hf_quartile_range,PREDICT_original_hf_entropy,PREDICT_original_logf_min_sigma1,PREDICT_original_logf_max_sigma1,...,"PREDICT_original_vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0",PREDICT_original_phasef_monogenic_min_WL3_N5,PREDICT_original_phasef_monogenic_max_WL3_N5,PREDICT_original_phasef_monogenic_mean_WL3_N5,PREDICT_original_phasef_monogenic_median_WL3_N5,PREDICT_original_phasef_monogenic_std_WL3_N5,PREDICT_original_phasef_monogenic_skewness_WL3_N5,PREDICT_original_phasef_monogenic_kurtosis_WL3_N5,PREDICT_original_phasef_monogenic_peak_WL3_N5,PREDICT_original_phasef_monogenic_peak_position_WL3_N5,PREDICT_original_phasef_monogenic_range_WL3_N5,PREDICT_original_phasef_monogenic_energy_WL3_N5,PREDICT_original_phasef_monogenic_quartile_range_WL3_N5,PREDICT_original_phasef_monogenic_entropy_WL3_N5,PREDICT_original_phasef_phasecong_min_WL3_N5,PREDICT_original_phasef_phasecong_max_WL3_N5,PREDICT_original_phasef_phasecong_mean_WL3_N5,PREDICT_original_phasef_phasecong_median_WL3_N5,PREDICT_original_phasef_phasecong_std_WL3_N5,PREDICT_original_phasef_phasecong_skewness_WL3_N5,PREDICT_original_phasef_phasecong_kurtosis_WL3_N5,PREDICT_original_phasef_phasecong_peak_WL3_N5,PREDICT_original_phasef_phasecong_peak_position_WL3_N5,PREDICT_original_phasef_phasecong_range_WL3_N5,PREDICT_original_phasef_phasecong_energy_WL3_N5,PREDICT_original_phasef_phasecong_quartile_range_WL3_N5,PREDICT_original_phasef_phasecong_entropy_WL3_N5,PREDICT_original_phasef_phasesym_min_WL3_N5,PREDICT_original_phasef_phasesym_max_WL3_N5,PREDICT_original_phasef_phasesym_mean_WL3_N5,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Liver-162_0,malignant,0.752802,0.067922,67.163862,11.220690,7.026174,3.458285,0.926487,0.039360,0.027479,0.008344,0.555713,0.075983,0.012185,0.003285,1.038624,0.014388,7455.448153,10739.958043,2627.406455,2133.440009,9.236892e+05,0.012883,1.073538,0.007832,0.666561,3.538946,1.593613,1.478897,0.702885,1.207853,2.496640,1.121624,10,2.872385,272683.457391,0.839124,17.551242,-0.287605,0.265794,...,15.389219,3.0,176.0,82.876996,78.76,51.979997,0.166057,-1.186755,42.96,12,173.0,1.923368e+09,90.0,17.293156,0.0,0.374992,0.040447,0.0,0.093533,3.006813,10.030828,0.0,0,0.374992,2086.907626,0.029188,15.263986,0.0,0.413396,0.044221,0.0,0.104164,2.824183,8.300528,0.0,0,0.413396,2573.519080,0.003259,15

In [ ]:
D_s.describe(include='all')

,label,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,PREDICT_original_sf_cvar_std_2.5D,PREDICT_original_sf_prax_avg_2.5D,PREDICT_original_sf_prax_std_2.5D,PREDICT_original_sf_evar_avg_2.5D,PREDICT_original_sf_evar_std_2.5D,PREDICT_original_sf_solidity_avg_2.5D,PREDICT_original_sf_solidity_std_2.5D,PREDICT_original_sf_area_avg_2.5D,PREDICT_original_sf_area_max_2.5D,PREDICT_original_sf_area_min_2.5D,PREDICT_original_sf_area_std_2.5D,PREDICT_original_sf_volume_2.5D,PREDICT_original_of_theta_x,PREDICT_original_of_theta_y,PREDICT_original_of_theta_z,PREDICT_original_hf_min,PREDICT_original_hf_max,PREDICT_original_hf_mean,PREDICT_original_hf_median,PREDICT_original_hf_std,PREDICT_original_hf_skewness,PREDICT_original_hf_kurtosis,PREDICT_original_hf_peak,PREDICT_original_hf_peak_position,PREDICT_original_hf_range,PREDICT_original_hf_energy,PREDICT_original_hf_quartile_range,PREDICT_original_hf_entropy,PREDICT_original_logf_min_sigma1,PREDICT_original_logf_max_sigma1,...,"PREDICT_original_vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0",PREDICT_original_phasef_monogenic_min_WL3_N5,PREDICT_original_phasef_monogenic_max_WL3_N5,PREDICT_original_phasef_monogenic_mean_WL3_N5,PREDICT_original_phasef_monogenic_median_WL3_N5,PREDICT_original_phasef_monogenic_std_WL3_N5,PREDICT_original_phasef_monogenic_skewness_WL3_N5,PREDICT_original_phasef_monogenic_kurtosis_WL3_N5,PREDICT_original_phasef_monogenic_peak_WL3_N5,PREDICT_original_phasef_monogenic_peak_position_WL3_N5,PREDICT_original_phasef_monogenic_range_WL3_N5,PREDICT_original_phasef_monogenic_energy_WL3_N5,PREDICT_original_phasef_monogenic_quartile_range_WL3_N5,PREDICT_original_phasef_monogenic_entropy_WL3_N5,PREDICT_original_phasef_phasecong_min_WL3_N5,PREDICT_original_phasef_phasecong_max_WL3_N5,PREDICT_original_phasef_phasecong_mean_WL3_N5,PREDICT_original_phasef_phasecong_median_WL3_N5,PREDICT_original_phasef_phasecong_std_WL3_N5,PREDICT_original_phasef_phasecong_skewness_WL3_N5,PREDICT_original_phasef_phasecong_kurtosis_WL3_N5,PREDICT_original_phasef_phasecong_peak_WL3_N5,PREDICT_original_phasef_phasecong_peak_position_WL3_N5,PREDICT_original_phasef_phasecong_range_WL3_N5,PREDICT_original_phasef_phasecong_energy_WL3_N5,PREDICT_original_phasef_phasecong_quartile_range_WL3_N5,PREDICT_original_phasef_phasecong_entropy_WL3_N5,PREDICT_original_phasef_phasesym_min_WL3_N5,PREDICT_original_phasef_phasesym_max_WL3_N5,PREDICT_original_phasef_phasesym_mean_WL3_N5,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
count,148,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,1.480000e+02,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,1.480000e+02,148.000000,148.000000,148.000000,148.000000,...,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,1.480000e+02,148.000000,148.000000,148.0,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.0,148.0,148.000000,148.000000,148.000000,148.000000,148.0,148.000000,148.000000,148.000000,148.000000

In [ ]:
D_s.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148 entries, Liver-162_0 to Liver-008_0
Columns: 494 entries, label to PREDICT_original_phasef_phasesym_entropy_WL3_N5
dtypes: float64(468), int64(25), object(1)
memory usage: 572.3+ KB


Below, we have splitted the Design set into a dataframe including the label, and a dataframe includinng only the information without label.

In [ ]:
D_s = load_D_set()
X_d, y_d = return_X_y(D_s)

In [ ]:
X_d.shape, y_d.shape

((148, 493), (148, 1))

### Cross validation

The cross validation function has been developed. This can be used for each 

In [ ]:
create_cross_val_sets(X_d,y_d)

44 Files have been created, in folder "datasets"
